In [2]:
import numpy as np
import scipy.stats
import scipy.io
import matplotlib.pyplot as plt
import math
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# Zero Crossing

def ZC(data): 
  n = len(data)
  count = 0
  
  for i in range(n-1):
    if (((data[i] > 0) and (data[i+1] < 0)) or ((data[i] < 0) and (data[i+1] > 0))):
      count = count + 1

  return count

In [ ]:
# Slope Sign Changes

def SSC(data): 
  n = len(data)
  count = 0
  
  for i in range(1, n-1):
    if (((data[i] < data[i+1]) and (data[i] < data[i-1])) or ((data[i] > data[i+1]) and (data[i] > data[i-1]))):
      count = count + 1

  return count

In [4]:
# Waveform Length

def WL(data):
  n = len(data)
  count = 0

  for i in range(n-1):
    count += np.abs(data[i+1]-data[i])

  return count


In [41]:
# Willison Amplitude

def WA(data, th):
  n = len(data)
  count = 0
  
  for i in range(n-1):
    if (abs(data[i+1] - data[i]) > th):
      count = count + 1

  return count


In [ ]:
# Variance 

def VAR(data):
    return np.var(data)

In [ ]:
# Skewness

def SKEW(data):
    return scipy.stats.skew(data)

In [ ]:
# Kurtosis

def KURT(data): 
    return scipy.stats.kurtosis(data)

In [ ]:
# Mean

def MEAN(data):
    return np.mean(data)

In [5]:
# Standard Deviation

def SD(data):
  return np.std(data)

In [28]:
def MVC(str):
    raw1 = np.loadtxt("filtereddata/" + str + "F1.csv", delimiter=',')
    raw2 = np.loadtxt("filtereddata/" + str + "F2.csv", delimiter=',')
    raw3 = np.loadtxt("filtereddata/" + str + "F3.csv", delimiter=',')
    raw4 = np.loadtxt("filtereddata/" + str + "E1.csv", delimiter=',')
    raw5 = np.loadtxt("filtereddata/" + str + "E2.csv", delimiter=',')
    raw6 = np.loadtxt("filtereddata/" + str + "E3.csv", delimiter=',')

    raws = np.abs(np.vstack((raw1, raw2, raw3, raw4, raw5, raw6)))

    mvcF = np.max(raws[:,1])
    mvcE = np.max(raws[:,3])

    return mvcF, mvcE

In [49]:
# Import
for who in ["Luigi", "James", "Dailyn"]:
    for what in ["Drill", "Softball", "Wood"]:
        for which in ["Physical", "Virtual"]:

          ww = who + "-" + which + "/"
          www = who + "-" + which + "/" + what
          wwwu = who + "_" + which + "_" + what

          raw = np.loadtxt("filtereddata/" + www + ".csv", delimiter=',')

          mvcF, mvcE = MVC(ww)

          fcu = np.divide(raw[:,1], mvcF)
          ecr = np.divide(raw[:,3], mvcE)

          if (who != "Dailyn"):
            segments = scipy.io.loadmat("segments/" + wwwu + ".mat")["segments"] # James, Luigi .mat files
          else:
            segments = np.loadtxt("segments/" + wwwu + ".csv", delimiter=',', dtype=int) # Dailyn .csv files
          
          starts = segments[:,0]
          ends = segments[:,1]

          WA_thresh = 0.06

          # Segment 
          features = np.empty((30,6))

          for i in range(30):
            start_ind = starts[i]
            end_ind = ends[i]
            trial_fcu = fcu[start_ind:end_ind]
            trial_ecr = ecr[start_ind:end_ind]

            wa_fcu = WA(trial_fcu, WA_thresh)
            wl_fcu = WL(trial_fcu)
            sd_fcu = SD(np.abs(trial_fcu))

            wa_ecr = WA(trial_ecr, WA_thresh)
            wl_ecr = WL(trial_ecr)
            sd_ecr = SD(np.abs(trial_ecr))

            features[i,0] = wa_fcu
            features[i,1] = wa_ecr

            features[i,2] = wl_fcu
            features[i,3] = wl_ecr

            features[i,4] = sd_fcu
            features[i,5] = sd_ecr


          out_string = wwwu + "_fn_features.csv"
          np.savetxt(out_string, features, delimiter=", ")

In [27]:
f = True

# Physical Virtual 
# Dailyn James Luigi
# Softball Drill Wood

for which in ["Physical", "Virtual"]:
    for who in ["Dailyn", "James", "Luigi"]:
        for what in ["Softball", "Drill", "Wood"]:
        
            www = who + "_" + which + "_" + what + "_fn_features.csv"
            raw = np.loadtxt(www, delimiter=',')

            reord = np.transpose(np.vstack((raw[:,1],raw[:,0],raw[:,3],raw[:,2],raw[:,5],raw[:,4])))
            
            if f: 
                all_trials = reord
                f = False
            else:
                all_trials = np.vstack((all_trials, reord))

header = "WA_ECR, WA_FCU, WL_ECR, WL_FCU, SD_ECR, SD_FCU"
np.savetxt("JamesFeaturesNorm.csv", all_trials, delimiter=", ", header=header)
